<a href="https://colab.research.google.com/github/lucasgstumpf/IA-backpropagation/blob/main/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

#https://www.dropbox.com/sh/cisaqxnki0rq5xz/AACNFkOgNd3VQ__Or4mAGjY_a/7º%20Semestre?dl=0&file_subpath=%2FInteligência+Artificial%2FProvas%2FIA%2FP2-IA.pdf&preview=Inteligência+Artificial.rar&subfolder_nav_tracking=1

##Manipulação do arquivo (Entrada)


In [ ]:
#Manipulação de arquivo de treinamento

df_treinamento = pd.read_csv("/content/treinamento.csv")
print(df_treinamento)
entrada = 3

     X1  X2  X3  X4  X5  X6  classe
0     1  19  35  28  17   4       1
1     4  22  38  32  14   9       1
2    -2  16  39  29  21   5       1
3    -2  22  33  32  22   1       1
4    -1  16  33  26  16   7       1
..   ..  ..  ..  ..  ..  ..     ...
822  13  74  63  76  82  66       5
823  18  81  60  79  84  72       5
824  19  73  64  84  83  74       5
825  16  77  66  75  86  66       5
826  20  83  58  84  85  67       5

[827 rows x 7 columns]


In [ ]:
df_treinamento = df_treinamento[["X1","X2","classe"]]

In [ ]:
df_treinamento = df_treinamento.iloc[0]

In [ ]:
df_treinamento = df_treinamento.to_frame().T


In [ ]:
df_treinamento = df_treinamento.values.tolist()

In [ ]:
df_treinamento

[[1, 19, 1]]

In [ ]:
entrada = df_treinamento[0][:-1] #Cada entrada vai ter mais ou menos isso aqui
classe_desejada = df_treinamento[0][-1]

In [ ]:
print(classe_desejada)

1


##Criação dos pesos

In [ ]:
# Defina as dimensões das camadas de sua rede
input_size = 2
hidden_sizes = [2]
output_size = 1

# Crie uma lista para armazenar os pesos
weights = []

# Gerando valores aleatórios para a primeira camada
weights.append(pd.DataFrame(np.random.rand(input_size, hidden_sizes[0]), columns=range(hidden_sizes[0])))

# Gerando valores aleatórios para as outras camadas ocultas
for i in range(1, len(hidden_sizes)):
    weights.append(pd.DataFrame(np.random.rand(hidden_sizes[i-1], hidden_sizes[i]), columns=range(hidden_sizes[i])))

# Gerando valores aleatórios para a camada de saída
weights.append(pd.DataFrame(np.random.rand(hidden_sizes[-1], output_size), columns=range(output_size)))



In [ ]:
weights[0][0].iloc[0] = 1.4
weights[0][0].iloc[1] = 1.3
weights[0][1].iloc[0] = 0.9
weights[0][1].iloc[1] = 1.2

weights[0].index = [2,3]

weights[1][0].iloc[0] = 0.5
weights[1][0].iloc[1] = 0.9

weights[0]

,0,1
2,1.4,0.9
3,1.3,1.2


##Processamento dos dados

In [ ]:
#weights é um array de data frame
n = 0.4

func_transferencia =  lambda x : x/2
func_transferencia_derivada = 0.5

def processa_rede(weights):
  #weights[0].index = entrada  #atualizo os indices da entrada

  for camada in range(len(weights)): #As camadas basicamente representam o meio entre os neuronios, ou seja, o  weights[0] representa os pesos entre a entrada e a primeira camada de neuronios
    indices = weights[camada].index.to_numpy() #Retorna um array com os indices
    colunas = weights[camada].columns.to_numpy() #Retorna um array com as colunas
    resultado = []
    for i in colunas:
      if(i != "erro"):
        valores_pesos = weights[camada][i].values #Pega todos os pesos de uma coluna
        multiplicacao = func_transferencia(valores_pesos * indices) 
        soma = np.sum(multiplicacao)
        resultado = np.append(resultado,soma)
    try:
      weights[camada+1].index = resultado
    except:
      print(f"resultado = {resultado}")
  return resultado[0]





In [ ]:
processa_rede(weights)

print(weights[0])
print(weights[1])

resultado = [2.0525]
     0    1
2  1.4  0.9
3  1.3  1.2
        0
3.35  0.5
2.70  0.9


#Calculo de erros

In [ ]:
for camada in range(len(weights)-1,-1,-1): #As camadas basicamente representam o meio entre os neuronios, ou seja, o  weights[0] representa os pesos entre a entrada e a primeira camada de neuronios
  weights[camada]['erro'] = pd.Series()

<ipython-input-121-f22286507a00>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  weights[camada]['erro'] = pd.Series()


In [ ]:

def ajusta_pesos(weights,erro_geral):
  primeira = True
  print("\n==INICIANDO AJUSTES DE PESO==\n")
  for camada in range(len(weights)-1,-1,-1):
    print(f"\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n-CAMADA: {camada}")
    print(f"    weights[camada]: \n {weights[camada]}\n\n")

    indices = weights[camada].index.to_numpy() #Retorna um array com os indices
    colunas = weights[camada].columns.to_numpy() #Retorna um array com as colunas
    for i in colunas:
      if(i != "erro"):
        print(f"    ----------\nCOLUNA: {i}")
        if(primeira):
          print(f"    -PRIMEIRA CAMADA SENDO PROCESSADA")
          #7 - calcula-se o erro dos neurônios da camada oculta
          erro_linha = weights[camada][i].values*erro_geral*func_transferencia_derivada #lembrar de por funcao
          weights[camada]["erro"] = erro_linha
          primeira = False
          #8 - os pesos da saída são ajustados
          print(f"    weights[camada][i] = {weights[camada][i].values}")
          print(f"    erro_geral = {erro_geral}")
          print(f"    indices = {indices}")
          weights[camada][i] = weights[camada][i] + (n*erro_geral) * indices
          print(f"    nweights[camada][i] = {weights[camada][i].values}")

          print(f"    \nweights[camada]: \n {weights[camada]}")
          
        else:
          
          print(f"    weights[camada]: \n {weights[camada][i]}\n\n")
          #9 - os pesos da camada oculta são ajustados
          erro_geral = weights[camada+1]["erro"].iloc[i]
          erro_linha = weights[camada][i].values*erro_geral #lembrar de por funcao
          weights[camada]["erro"] = erro_linha
          primeira = False
          #8 - os pesos da saída são ajustados
          print(f"    weights[{camada}][{i}] = {weights[camada][i].values}") # O ERRO TA AQUI
          print(f"    erro_geral = {erro_geral}")
          print(f"    indices = {indices}")
          print(f"    n = {n}")
          weights[camada][i] = weights[camada][i] + (n* erro_geral) * indices
          print(f"    weights[{camada}][{i}] = {weights[camada][i].values}")

          print(f"    \nweights[camada]: PROCESSADA\n {weights[camada]}")
  return weights




In [ ]:
resultado = processa_rede(weights)

resultado = [0.03705019]


In [ ]:
erro_geral = (classe_desejada - resultado) * func_transferencia_derivada
erro_geral

0.48147490679687477

In [ ]:
weights = ajusta_pesos(weights,erro_geral)


In [ ]:
erro = True
valorErroEntrada = 0.1
numeroIteracoes = False
numeroIteracoesInt = 500
i = 0
while(True):
  resultado = processa_rede(weights)
  erro_geral = (classe_desejada - resultado) * func_transferencia_derivada
  weights = ajusta_pesos(weights,erro_geral)

  if(erro == True) and (erro_geral < valorErroEntrada):
    print("Resultado: ", resultado)
    break
  if numeroIteracoes == True and numeroIteracoesInt < i:
    print("Resultado: " + resultado)
    break

  i = i + 1

resultado = [1.00166037]

==INICIANDO AJUSTES DE PESO==


@@@@@@@@@@@@@@@@@@@@@@@@@@@@
-CAMADA: 1
    weights[camada]: 
                  0      erro
2.798348  0.236907  0.000248
2.161425  0.620134  0.000655


    ----------
COLUNA: 0
    -PRIMEIRA CAMADA SENDO PROCESSADA
    weights[camada][i] = [0.23690667 0.620134  ]
    erro_geral = -0.0008301871320843812
    indices = [2.79834841 2.16142536]
    nweights[camada][i] = [0.23597741 0.61941624]
    
weights[camada]: 
                  0      erro
2.798348  0.235977 -0.000098
2.161425  0.619416 -0.000257

@@@@@@@@@@@@@@@@@@@@@@@@@@@@
-CAMADA: 0
    weights[camada]: 
           0         1      erro
2  1.230261  0.734285  0.000481
3  1.045392  0.951427  0.000623


    ----------
COLUNA: 0
    weights[camada]: 
 2    1.230261
3    1.045392
Name: 0, dtype: float64


    weights[0][0] = [1.23026105 1.04539157]
    erro_geral = -9.833843404326927e-05
    indices = [2 3]
    n = 0.4
    weights[0][0] = [1.23018238 1.04527357]
    
weights[ca